<a href="https://colab.research.google.com/github/Nickmescher/ml5sem/blob/main/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
transactions = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/transactions.csv", nrows=1000000)
print(transactions)
tr_mcc_codes = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tr_mcc_codes.csv", sep=";")
print(tr_mcc_codes)
tr_types = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tr_types.csv", sep=";")
print(tr_types)
gender_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/gender_train.csv", sep=",")
print(gender_train)

        customer_id   tr_datetime  mcc_code  tr_type    amount   term_id
0          39026145    0 10:23:26      4814     1030  -2245.92       NaN
1          39026145    1 10:19:29      6011     7010  56147.89       NaN
2          39026145    1 10:20:56      4829     2330 -56147.89       NaN
3          39026145    1 10:39:54      5499     1010  -1392.47       NaN
4          39026145    2 15:33:42      5499     1010   -920.83       NaN
...             ...           ...       ...      ...       ...       ...
999995     98801178  269 10:29:06      5921     1010 -23016.59    127873
999996     98801178  270 10:26:45      5211     1010 -61380.88  10114209
999997     98801178  270 15:18:53      5921     1010 -11800.27  10105341
999998     98801178  271 12:47:13      5211     1010 -16170.59  10114209
999999     98801178  271 14:51:56      5921     1010 -10025.77    123991

[1000000 rows x 6 columns]
     mcc_code                                    mcc_description
0         742                  


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [ ]:
tr_type_SAMPLE = transactions.tr_type.sample(n=1000)

tr_sample = (tr_types['tr_description'].str.contains('АТМ') | tr_types['tr_description'].str.contains('POS'))
tr_sample = pd.DataFrame(tr_sample)
tr_sample['tr_type'] = tr_types['tr_type']
tr_sample = tr_sample.loc[tr_sample['tr_description'] == True]

quan = tr_type_SAMPLE.isin(tr_sample['tr_type']).value_counts()[True]

print("Доля транзакций", quan/1000)

Доля транзакций 0.635


## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [ ]:
freq = transactions['tr_type'].value_counts()
freq10 = freq.head(10)
freq1 = pd.DataFrame({'tr_type': tr_types[tr_types['tr_type'].isin(freq10.index.values)].tr_type, 'tr_description': tr_types[tr_types['tr_type'].isin(freq10.index.values)].tr_description, 'value_counts': freq10.values},)
print(freq1)

     tr_type                                     tr_description  value_counts
39      7010       Взнос наличных через АТМ (в своем тер.банке)        231117
47      7030  Перевод на карту (с карты) через АТМ (в предел...        151166
59      7070  Перевод на карту (с карты) через Мобильный бан...        149006
60      7071  Перевод на карту (с карты) через Мобильный бан...        137658
98      1010                              Покупка. POS ТУ СБ РФ        118975
99      1030                     Оплата услуги. Банкоматы СБ РФ         49830
100     1100                                Покупка. ТУ  Россия         28972
101     1110                             Покупка. POS ТУ Россия         26078
106     2010              Выдача наличных в АТМ Сбербанк России         15781
129     2370  Списание с карты на карту по операции <перевод...         15476


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [ ]:
cust_amoun = transactions[['customer_id', 'amount']]

cust_amoun_pos = cust_amoun.loc[cust_amoun['amount'] > 0].groupby('customer_id').amount.sum()

cust_amoun_neg = cust_amoun.loc[cust_amoun['amount'] < 0].groupby('customer_id').amount.sum()

idmax = cust_amoun_pos.idxmax()
idmin = cust_amoun_neg.idxmin()
print(idmax, idmin)

diff = abs(cust_amoun_pos.max() + cust_amoun_neg.min())
print(diff)

# data = {'customer_id': [],
#         'amount_sum':[]}

# amount_table = pd.DataFrame(data)

# for key, group in cust_amoun_gr:
#     amount_pos = 0
#     amount_neg = 0
#     for x in group.amount.values:
#         if x > 0:
#             amount_pos += x
#         else:
#             amount_neg += x
#     new_row1 = {'customer_id': key, 'amount_sum': amount_pos}
#     new_row2 = {'customer_id': key, 'amount_sum': amount_neg}
#     amount_table = amount_table.append(new_row1, ignore_index=True)
#     amount_table = amount_table.append(new_row2, ignore_index=True)

# print(amount_table)
# max_amount = amount_table['amount_sum'].max()
# max_amount_id = amount_table['amount_sum'].idxmax()
# max_amount_id = int(amount_table.iloc[max_amount_id].customer_id)
# print(max_amount_id)

# min_amount = amount_table['amount_sum'].min()
# min_amount_id = amount_table['amount_sum'].idxmin()
# min_amount_id = int(amount_table.iloc[min_amount_id].customer_id)
# print(min_amount_id)

# print(abs(max_amount + min_amount))

70780820 70780820
1837317.980000019


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [ ]:
top_10 = transactions[['tr_type', 'amount']]
mean1 = top_10[top_10['tr_type'].isin(freq10.index.values)].groupby('tr_type').amount.mean()
median = top_10[top_10['tr_type'].isin(freq10.index.values)].groupby('tr_type').amount.median()

print(mean1, median)

# data = {'tr_types': [],
#         'mean': [],
#         'meadian': []}

# mean_median = pd.DataFrame(data)

# for key, group in top_10:
#     med = np.array(group.amount.values)
#     new_row = {'tr_types': key, 'mean': group.amount.values.mean(), 'meadian': np.median(med)}
#     mean_median = mean_median.append(new_row, ignore_index=True)
# print(mean_median)

tr_type
1010    -19784.748640
1030     -5320.980222
1100    -44061.827262
1110    -32119.330371
2010   -136077.629325
2370   -205418.249032
7010    276391.789596
7030     86104.332909
7070     65569.831700
7071     66806.826623
Name: amount, dtype: float64 tr_type
1010     -7411.52
1030     -2245.92
1100    -10188.26
1110    -11207.57
2010    -44918.32
2370    -44918.32
7010    112295.79
7030     13951.52
7070     11319.42
7071      3593.47
Name: amount, dtype: float64


In [ ]:
# all_types = transactions.loc[transactions['customer_id'] == max_amount_id]
# all_types_gr = all_types.groupby('tr_type')

# data = {'tr_types': [],
#         'mean': [],
#         'meadian': []}

# mean_median = pd.DataFrame(data)

# for key, group in all_types_gr:
#     med = np.array(group.amount.values)
#     new_row = {'tr_types': key, 'mean': group.amount.values.mean(), 'meadian': np.median(med)}
#     mean_median = mean_median.append(new_row, ignore_index=True)
# print(mean_median)

    tr_types          mean      meadian
0     2010.0 -2.941056e+06 -3368873.660
1     2330.0 -2.382398e+06 -2245915.770
2     2370.0 -2.218505e+06 -2245915.770
3     6110.0  1.756293e+04    10028.010
4     7020.0  1.465129e+04    10319.985
5     7021.0  2.850067e+04    28500.670
6     7030.0  1.406196e+04     8848.910
7     7031.0  1.921036e+04    12992.620
8     7034.0  1.430648e+04    14306.480
9     7040.0  2.153833e+04    14396.320
10    7070.0  1.384821e+04     8714.150
11    7071.0  1.860057e+04    12543.440
12    7074.0  2.281159e+04     9387.930


## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [ ]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions.shape

(999584, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [ ]:
neg_transactions = transactions.loc[transactions['amount'] < 0]
neg_transactions_gr = neg_transactions.groupby('gender')

pos_transactions = transactions.loc[transactions['amount'] > 0]
pos_transactions_gr = pos_transactions.groupby('gender')

print(abs(neg_transactions_gr.amount.mean()[0] - neg_transactions_gr.amount.mean()[1]))
print(abs(pos_transactions_gr.amount.mean()[0] - pos_transactions_gr.amount.mean()[1]))

32718.054920224655
63366.57104801464


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income". 
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [ ]:
pos_transactions_male = transactions.loc[(transactions['amount'] > 0) & (transactions['gender'] == 0)]
pos_transactions_male_gr = pos_transactions_male.groupby('tr_type')
max_income = pos_transactions_male_gr.amount.mean().sort_values()
max_income_male = max_income.head(10)
print(max_income_male)

pos_transactions_female = transactions.loc[(transactions['amount'] > 0) & (transactions['gender'] == 1)]
pos_transactions_female_gr = pos_transactions_female.groupby('tr_type')
max_income = pos_transactions_female_gr.amount.sum().sort_values()
max_income_female = max_income.head(10)
print(max_income_female)

common_f = pd.DataFrame({'tr_type': max_income_female.index, 'amount': max_income_female.values})
common_m = pd.DataFrame({'tr_type': max_income_male.index, 'amount': max_income_male.values})

common = pd.merge(common_m, common_f, how="inner", left_on='tr_type', right_on='tr_type')
print(common)
# for a, b in zip(max_income_male.index.values, max_income_female.index.values):
#   if a == b:
#     print(a)

tr_type
4051     1122.960000
4210     2245.920000
4110     2245.920000
2370     2335.750000
4100     5041.860000
7075     6737.750000
1010    15766.330000
2110    22459.160000
6200    27212.886321
2010    33688.740000
Name: amount, dtype: float64
tr_type
2020      2245.92
4110      4491.84
1010     35317.03
8100     64682.37
6000     79280.83
7041     87590.72
2370    113194.15
7015    121279.45
2110    179673.26
2010    224591.59
Name: amount, dtype: float64
   tr_type  amount_x   amount_y
0     4110   2245.92    4491.84
1     2370   2335.75  113194.15
2     1010  15766.33   35317.03
3     2110  22459.16  179673.26
4     2010  33688.74  224591.59


In [ ]:
day = transactions['tr_datetime'].str.split(' ', expand=True)
day.columns = ['tr_day', 'tr_time']
day = pd.merge(day, transactions,  how="inner", left_on=day.index, right_on=transactions.index)
day['tr_day'] = day['tr_day'].astype(int)
day = day.loc[day['amount']>0]
day = pd.merge(day, common, how="inner")
day = day.loc[day['tr_day'] > 449]
days = day.sort_values(by='tr_day').groupby(['tr_day', 'tr_type']).amount.mean()
days = pd.DataFrame(days)
# days.columns = ['tr_day', 'tr_type', 'amount']
import matplotlib.pyplot as plt
# daysplt = np.array([days.index[0][0], days.index[1][0], days.index[2][0]])
# typesplt = np.array([days.index[0][1], days.index[1][1], days.index[2][1]])
# plot1 = plt.plot(daysplt, days.amount.values)
print(days)
# print(daysplt)


                 amount
tr_day tr_type         
456    1010     6513.16


## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [ ]:
transactions_male = transactions.loc[(transactions['amount'] < 0) & (transactions['gender'] == 0)]
transactions_male_gr = transactions_male.groupby('mcc_code').amount.sum()
print(transactions_male_gr)

transactions_female = transactions.loc[(transactions['amount'] < 0) & (transactions['gender'] == 1)]
transactions_female_gr = transactions_female.groupby('mcc_code').amount.sum()
print(transactions_female_gr)

difference1 = pd.DataFrame({'mcc_code':transactions_female_gr.index, 'female_am':transactions_female_gr.values})
difference2 = pd.DataFrame({'mcc_code':transactions_male_gr.index, 'male_am':transactions_male_gr.values})
difference = pd.merge(difference1, difference2, how="inner")
difference = pd.merge(difference, tr_mcc_codes, how="inner")
difference['diff'] = abs(abs(difference.male_am) - abs(difference.female_am))
print(difference.sort_values(by='diff', ascending=False).head(10)[['diff', 'mcc_code', 'mcc_description']])

mcc_code
742      -386207.68
1711     -674897.69
1799     -261626.73
2741      -20746.42
3000   -74431645.27
           ...     
8699     -521107.48
8999   -19432057.92
9222     -361143.24
9311     -406197.45
9399    -1481389.65
Name: amount, Length: 174, dtype: float64
mcc_code
742      -132277.93
1711     -206264.90
1731     -121542.67
1799    -1129257.68
2741     -221290.08
           ...     
8999   -28643346.63
9222     -988205.17
9311     -373271.22
9399    -1810820.67
9402       -4581.67
Name: amount, Length: 175, dtype: float64
             diff  mcc_code                                    mcc_description
116  3.085862e+09      6011  Финансовые институты — снятие наличности автом...
21   2.499153e+09      4829                                  Денежные переводы
115  2.208580e+08      6010   Финансовые институты — снятие наличности вручную
62   1.554326e+08      5541                            Станции техобслуживания
59   7.352877e+07      5511  Легковой и грузовой транспорт — пр

## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов. 